In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import dateutil.parser

In [2]:
# dictionary of text dates and links (August 2016 to November 2016)

date1 = '2016-08-10' 
url1 = 'https://www.reuters.com/article/us-samsung-elec-m-a-dacor-idUSKCN10L2RR'

date2 = '2016-08-22'
url2 = 'https://www.reuters.com/article/us-samsung-elec-phones-refurbishment-idUSKCN10X0FT'

date3 = '2016-08-24'
url3 = 'https://www.reuters.com/article/us-samsung-elec-tencent-holdings-stocks-idUSKCN10Z0BJ'

date4 = '2016-08-31'
url4 = 'https://www.reuters.com/article/us-nokia-corp-managementchanges-idUSKCN1160I8'


date5 = '2016-09-02'
url5 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-replacement-idUSKCN1180TV'

date6 = '2016-09-06'
url6 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKCN11C0CV'

date7 = '2016-09-11'
url7 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-stocks-idUSKCN11I02M'

date8 = '2016-09-15'
url8 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKCN11L2M1'

date7 = '2016-09-19'
url7 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-china-idUSKCN11P0J9'

date8 = '2016-09-30'
url8 = 'https://www.reuters.com/article/us-samsung-china-idUSKCN1200LD'

date9 = '2016-11-03'
url9 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKBN12Y0T7'


date10 = '2016-11-05'
url10 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKBN13101Q'

date11 = '2016-11-14'
url11 = 'https://www.reuters.com/article/us-harman-intl-ind-samsung-elec-idUSKBN1390O9'

date12 = '2016-11-20'
url12 = 'https://www.reuters.com/article/us-samsung-elec-recall-brand-idUSKBN13F146'

date13 = '2016-11-22'
url13 = 'https://www.reuters.com/article/us-southkorea-politics-idUSKBN13I049'

date14 = '2016-11-28'
url14 = 'https://www.reuters.com/article/us-samsung-elec-restructuring-idUSKBN13N2GS'



url_dict = {date1:url1,
            date2:url2,
            date3:url3,
            date4:url4,
            date5:url5,
            date6:url6,
            date7:url7,
            date8:url8,
            date9:url9,
            date10:url10,
            date11:url11,
            date12:url12,
            date13:url13,
            date14:url14,
           }


In [3]:
# extract info from each website and store it in dataframe
df_text = pd.DataFrame(columns = ['Date','Header','Text'])
header = []
text = []
date = []
for key in url_dict:
    page = requests.get(url_dict[key]).text
    soup = BeautifulSoup(page, 'html.parser')
    date.append(datetime.strptime(key, '%Y-%m-%d').date())
    url_header = soup.find('h1').text
    header.append(url_header)
    url_text = soup.find('div', class_ ='ArticleBodyWrapper')
    url_text = url_text.text
    text.append(url_text)
df_text['Date']=date
df_text['Header']=header
df_text['Text']=text

# remove the author and "min read" sections
df_text_temp = [] 
for text in df_text['Text']:
    loc = text.find('(Reuters)')
    df_text_temp.append(text[loc+9:])
df_text['Text'] = df_text_temp




In [4]:
df_text['Date'] = df_text['Date'].apply(lambda x: pd.to_datetime(x))
df_text.head()

,Date,Header,Text
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi..."
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...


In [5]:
# import stock data of netflix
df_stock = pd.read_csv('Samsung.csv', header=0)

df_stock['Date']=df_stock['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


df_stock = df_stock.drop(['High','Low','Close','Adj Close'], axis=1)
df_stock.head()

,Date,Open,Volume
0,2016-01-04,25200.0,15346950
1,2016-01-05,24040.0,10800100
2,2016-01-06,24160.0,18337600
3,2016-01-07,23320.0,14119400
4,2016-01-08,23260.0,12888150


In [6]:
# Merge two dataframes
df = pd.merge(df_text, df_stock, how='inner', on='Date')
df = df.sort_values('Date')

In [7]:
# show % change in stock price after news article publish
pct_stock = df['Open'].pct_change()*100
df['% Change in Stock Price'] = pct_stock
pct_volume = df['Volume'].pct_change()*100
df['% Change in Volume'] = pct_volume
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639


# Processing the text


## import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [8]:
# import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [9]:

text_nlp = nlp(df['Text'][1]) 
# Remove stop words
text_nlp_clean = [word for word in text_nlp if word.is_stop == False]
# Lemmatize words
text_nlp_clean = [word.lemma_ for word in text_nlp_clean]

In [10]:
text_clean = []
for ind, row in df.iterrows():
    
    # lower casing
    text_lower = row['Text'].lower()
    
    # tokenization
    text_nlp = nlp(row['Text']) 
    
    # remove punctuation and empty space
    text_nlp_clean = [word for word in text_nlp if not word.is_punct | word.is_space]
    
    # Remove stop words
    text_nlp_clean = [word for word in text_nlp_clean if word.is_stop == False]
    
    # Lemmatize tokens
    text_nlp_clean = [word.lemma_ for word in text_nlp_clean]
    
    # add preprocessed text to list
    text_clean.append(text_nlp_clean)

# turn list of strings to string
to_string = []
for text in text_clean:
    to_string.append(' '.join(text))


df['Text (clean)'] = to_string
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean)
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN,south korean tech giant Samsung Electronics Co...
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936,Samsung Electronics Co Ltd plan launch program...
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066,Tencent Holdings Ltd 0700.hk Samsung Electroni...
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003,head Nokia small highly profitable patent lice...
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639,Samsung Electronics Co Ltd recall Galaxy Note ...


# Using a Pretrained Model 

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [12]:
# a function to return sentiment score
def polarity_score(text):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(text)['compound']

vader_score = []
for text in df['Text (clean)']:
    score = polarity_score(text)
    vader_score.append(score)

df['Vader Sentiment Score'] = vader_score

In [13]:
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean),Vader Sentiment Score
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN,south korean tech giant Samsung Electronics Co...,0.8555
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936,Samsung Electronics Co Ltd plan launch program...,0.9971
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066,Tencent Holdings Ltd 0700.hk Samsung Electroni...,0.9963
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003,head Nokia small highly profitable patent lice...,0.9930
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639,Samsung Electronics Co Ltd recall Galaxy Note ...,0.9854


## Sentiment Analysis

In [14]:
# run this cell to import nltk
import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from utils import process_tweet, build_freqs

In [15]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:

# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [17]:
# split the data into two pieces (80-20), one for training and one for testing (validation set)  
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [18]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [19]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [20]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11340


In [21]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [22]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

In [23]:
Y.shape

(8000, 1)

### SVM Parameters

In [24]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, Y)

C:\Users\aashi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(random_state=0)

In [25]:

# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [26]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet_prob(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict_proba(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [27]:

my_tweet = 'The plot was terrible and I was sad until the ending!'
predict_tweet(my_tweet, freqs)

array([0.])

# Testing the Model 

In [28]:
sentiment = []
sentiment_prob = []
for i in df.iloc[:,2]:
    sentiment.append(predict_tweet(i, freqs)[0])
    sentiment_prob.append(predict_tweet_prob(i, freqs)[0][1])

In [29]:
sentiment


[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [30]:
sentiment_prob

[0.6695515692647688,
 0.012659091349997681,
 0.8326791209409139,
 0.07580211607174459,
 0.0025454909101454495,
 0.018872802031474317,
 0.14695255146151398,
 0.22099345466835518,
 0.383674231977853,
 0.012820320610267732,
 0.01687993067573554,
 0.009441774949025096]

In [31]:
df['sentiment'] = sentiment
df['sentiment_prob'] = sentiment_prob
df

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean),Vader Sentiment Score,sentiment,sentiment_prob
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN,south korean tech giant Samsung Electronics Co...,0.8555,1.0,0.669552
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936,Samsung Electronics Co Ltd plan launch program...,0.9971,0.0,0.012659
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066,Tencent Holdings Ltd 0700.hk Samsung Electroni...,0.9963,1.0,0.832679
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003,head Nokia small highly profitable patent lice...,0.9930,0.0,0.075802
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639,Samsung Electronics Co Ltd recall Galaxy Note ...,0.9854,0.0,0.002545
5,2016-09-06,Samsung trips on quality control in rush to pi...,- In its rush to beat rival products to marke...,32140.0,7660150,0.752351,-6.045014,rush beat rival product market notably Apple a...,0.8807,0.0,0.018873
6,2016-09-19,Samsung says China phone fire caused by extern...,- Samsung Electronics Co Ltd 005930.KS said o...,30760.0,22408150,-4.293715,192.528867,Samsung Electronics Co Ltd 005930.ks say Monda...,-0.9287,0.0,0.146953
7,2016-09-30,Samsung slammed by Chinese state TV over Note ...,- Chinese state broadcaster CCTV has slammed ...,31800.0,11762700,3.381014,-47.507045,chinese state broadcaster CCTV slam south kore...,0.4588,0.0,0.220993
8,2016-11-03,Samsung Electronics reviewing ways to limit No...,- Samsung Electronics Co Ltd said on Thursday...,32600.0,10341850,2.515723,-12.079285,Samsung Electronics Co Ltd say Thursday review...,0.1027,0.0,0.383674
9,2016-11-14,Samsung to buy car tech company Harman for $8 ...,- Samsung Electronics has agreed to buy Harma...,31900.0,15133300,-2.147239,46.330686,Samsung Electronics agree buy Harman Internati...,0.9914,0.0,0.012820


In [39]:
# Converting % stock price change to % stock price change per day
days = [0]
for i in range(len(df['Date'])):
    if i != 0:
        date_diff = df['Date'][i]-df['Date'][i-1]
        day = date_diff.days
        days.append(day)
print(days)
df['Days Elapsed'] = days
df.head()
df['% Change in Stock Price Per Day'] = df['% Change in Stock Price']/df['Days Elapsed']
df.head()

[0, 12, 2, 7, 2, 4, 13, 11, 34, 11, 8, 6]


,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean),Vader Sentiment Score,sentiment,sentiment_prob,Days Elapsed,% Change in Stock Price Per Day
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN,south korean tech giant Samsung Electronics Co...,0.8555,1.0,0.669552,0,NaN
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936,Samsung Electronics Co Ltd plan launch program...,0.9971,0.0,0.012659,12,0.569028
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066,Tencent Holdings Ltd 0700.hk Samsung Electroni...,0.9963,1.0,0.832679,2,0.179211
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003,head Nokia small highly profitable patent lice...,0.9930,0.0,0.075802,7,-0.331633
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639,Samsung Electronics Co Ltd recall Galaxy Note ...,0.9854,0.0,0.002545,2,-1.401584


In [41]:
df['Vader Sentiment'] = df['Vader Sentiment Score'].apply(lambda x: "positive" if x>0 else "negative")
df['Logistic Regression Sentiment'] = df['Vader Sentiment Score'].apply(lambda x: "positive" if x==1 else "negative")

In [ ]:
df_summary = df